In [1]:
import numpy as np
import torch
import math
import torchvision
from torch import nn
from torch.nn import functional as F
from torchvision import datasets, transforms
import warnings
from functools import partial

/data3/hop20001/.conda/envs/torchenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Test partial function
relu_inplace = partial(F.relu, inplace = True)
batch_size, dim1, dim2 = 2, 2, 2
shape = [batch_size, dim1, dim2]
in_tensor = torch.Tensor(*shape)
nn.init.uniform_(in_tensor, a = -1, b = 1)
print(in_tensor)
out_tensor = relu_inplace(in_tensor)
print(out_tensor)

tensor([[[ 0.8832, -0.4831],
         [ 0.2317,  0.8681]],

        [[-0.1631,  0.4590],
         [ 0.0352,  0.1240]]])
tensor([[[0.8832, 0.0000],
         [0.2317, 0.8681]],

        [[0.0000, 0.4590],
         [0.0352, 0.1240]]])


In [4]:
# test ReLU_masked model
from models_util import *
batch_size, dim1, dim2 = 1, 1, 2
shape = [batch_size, dim1, dim2]
in_tensor = torch.Tensor(*shape)
nn.init.uniform_(in_tensor, a = -1, b = 1)
print(in_tensor)
model = ReLU_masked()
for i in range(2):
    in_tensor = model(in_tensor)
print(in_tensor)
for p in model.parameters():
    print(p.name, p.data)
model.init = 0
print('Number of features', model.num_feature)
for i in range(4):
    print('Currently used features', model.current_feature)
    # print(eval('model.alpha_aux_{}_0'.format(model.current_feature)))
    in_tensor = model(in_tensor)
print(in_tensor)
for p in model.parameters():
    print(p.name, p.data)

tensor([[[-0.6670,  0.6534]]])
tensor([[[0.0000, 0.6534]]], grad_fn=<AddBackward0>)
None tensor([[0.0763, 0.9488]])
None tensor([[0.5329, 0.3484]])
Number of features 2
Currently used features 0
Current used:  Parameter containing:
tensor([[0.0763, 0.9488]], requires_grad=True)
Currently used features 1
Current used:  Parameter containing:
tensor([[0.5329, 0.3484]], requires_grad=True)
Currently used features 0
Current used:  Parameter containing:
tensor([[0.0763, 0.9488]], requires_grad=True)
Currently used features 1
Current used:  Parameter containing:
tensor([[0.5329, 0.3484]], requires_grad=True)
tensor([[[0.0000, 0.6534]]], grad_fn=<AddBackward0>)
None tensor([[0.0763, 0.9488]])
None tensor([[0.5329, 0.3484]])


In [2]:
from models_util import *
from models_cifar import *

model_name = 'ResNet18'
model = ResNet18()
# def get_my_code(base):

#     class model_build(base, model_util):
#         def __init__(self,):
#           pass
#     return model_build
# # model_build(eval(model_name), model_util)
# model_Masked = get_my_code(model)
config = 1
model_util_new(model, config)

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F


# Define your model
model = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Sequential(
        nn.Linear(10, 10),
        nn.ReLU()
    ),
    nn.Linear(10, 10)
)

# Generate some random input data
input_data = torch.randn(1, 10)

# Choose your replacement activation function
replacement_fn = nn.Sigmoid()

# Replace all ReLU functions with the replacement function
def replace_relu(model):
    for name, module in model.named_children():
        if isinstance(module, nn.ReLU):
            model._modules[name] = replacement_fn
        else:
            replace_relu(module)

replace_relu(model)
print(model)
# Run the model
output = model(input_data)

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): Sigmoid()
  (2): Sequential(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Sigmoid()
  )
  (3): Linear(in_features=10, out_features=10, bias=True)
)


In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F


class model_v2(nn.Module):
    def __init__(self):     
        super(model_v2, self).__init__()
        self.stacks = nn.Sequential(
        nn.Linear(10, 10),
        nn.ReLU(),
        nn.Sequential(
            nn.Linear(10, 10),
            nn.ReLU()
        ),
        nn.Linear(10, 10)
        )
    def forward(self, x):
        out = self.stacks(x)
        out = F.relu(out)

model = model_v2()

# Generate some random input data
input_data = torch.randn(1, 10)

# Choose your replacement activation function
replacement_fn = nn.Sigmoid()

# Replace all ReLU functions with the replacement function
def replace_relu(model):
    for name, module in model.named_children():
        if isinstance(module, nn.ReLU):
            model._modules[name] = replacement_fn
        else:
            replace_relu(module)

replace_relu(model)
print(model)
# Run the model
output = model(input_data)

model_v2(
  (stacks): Sequential(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Sigmoid()
    (2): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): Sigmoid()
    )
    (3): Linear(in_features=10, out_features=10, bias=True)
  )
)
